In [11]:
import pandas as pd
import time
import datetime
import numpy as np
import pandas_ta as ta

In [12]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [13]:
def read_data(path,interval):
    raw_data = pd.read_csv(path,index_col=0,parse_dates=True)
    agg_dict = {'open':'Open','high':'High','low':'Low','close':'Close'}
    df = raw_data['ltp'].resample(interval).ohlc().dropna().rename(columns=agg_dict)
    symbol = pd.unique(raw_data['symbol'])[0]
    df.insert(0,'symbol',symbol)
    return df

~~~python
target_pt = 10
stoploss_pt = 10
trail = 5

sl_hit = False
target_hit = False
trade = 1

entry = data['Close'][0]
SL = data['Close'][0] - stoploss_pt
target = entry + target_pt
trailing_SL = entry + trail
print('Trade : {:<3} | Entry Price: {:<8} | target : {:<8} [{:>2}] | SL : {:<8} [{:>2}] | trail : {:<6}\n'.format(trade,entry,target,target_pt,SL,stoploss_pt,trail))



for i in range(0,len(data)):

    if data['Close'][i] <= SL and sl_hit == False and target_hit == False: 
        print("{:<3} | close : {:<8} | entry : {:<8} | SL HIT : {:<8}\n".format(i,data['Close'][i],entry,SL))
        entry = 0
        stoploss = 0
        trailing_SL = 0
        target = 0
        SL = 0
        sl_hit = True
        continue
        
    if data['Close'][i] >= target and sl_hit == False and target_hit == False: 
        print("{:<3} | close : {:<8} | entry : {:<8} | target HIT : {:<8}\n".format(i,data['Close'][i],entry,SL))
        entry = 0
        stoploss = 0
        trailing_SL = 0
        target = 0
        SL = 0
        target_hit = True
        continue
        
    if sl_hit == False and target_hit == False:
    
       
        
        if data['Close'][i] > trailing_SL:
            updated_sl = trailing_SL
            SL = updated_sl
            trailing_SL = trailing_SL+ trail
            print('{:<3} | close : {:<8} | entry : {:<8} | new SL : {:<8} | TSL : {:<8} | Target : {:<8}'.format(i,data['Close'][i],entry,updated_sl,trailing_SL,target))
        else:
            print('{:<3} | close : {:<8} | entry : {:<8} | SL     : {:<8} | TSL : {:<8} | Target : {:<8}'.format(i,data['Close'][i],entry,SL,trailing_SL,target))
        
    
    if sl_hit == True or target_hit == True:
        target_pt = 10
        stoploss_pt = 10
        trail = 5
    
        entry = data['Close'][i]
        SL = data['Close'][i] - stoploss_pt
        target = entry + target_pt
        
        trailing_SL = entry + trail
        trade +=1
        print('Trade : {:<3} | entry Price: {:<8} | target : {:<8} [{:>2}] | SL : {:<8} [{:>2}] | trail : {:<6}\n'.format(trade,entry,target,target_pt,SL,stoploss_pt,trail))
        sl_hit = False
        target_hit = False
~~~

In [14]:
def macd(data,fast=12,slow=26,signal=9):
    data = ta.macd(data['Close'],fast=fast,slow=slow,signal=signal)
    data.rename(columns={data.columns[0]:'macd',data.columns[1]:'histogram',data.columns[2]:'macd_signal'},inplace=True)
    data = data[['macd','macd_signal']]
    return data

def supertrend(data,length=10,multiplier=1):
    data = ta.supertrend(high=data['High'],low=data['Low'],close=data['Close'],length=length,multiplier=multiplier)
    data.rename(columns={data.columns[0]:'supertrend'},inplace=True)
    data = data.loc[:,[data.columns[0]]]
    return data

def combine_macd_st(price_data,macd_data,supertrend_data):
    data = pd.concat([price_data,macd_data,supertrend_data],axis=1)
    return data

#combine ST and macd
def strategy_technical_indicator(data,fast=12,slow=26,signal=9,length=10,multiplier=1):
    df_macd = macd(data=data,fast=fast,slow=slow,signal=signal)
    df_supertrend = supertrend(data=data,length=length,multiplier=multiplier)
    final_df = combine_macd_st(data[['symbol','Close']],df_macd,df_supertrend)
    final_df = final_df.round(2)
    return final_df

In [21]:
def st_crossover(data,stoploss_pt,target_pt,trailing_stoploss_pt):
    
    data['position'] = 0
    data['signal'] = 0
    data['trade_price'] = 0
    data['trade_return'] = 0

    
    for i in range(0,len(data)):
        if data.index[i].time() <= datetime.time(15,0) and data.index[i].time() >= datetime.time(9,30):
            ##### long position
            if data['position'][i-1] == 1:
                data['position'].iloc[i] = data['position'].iloc[i-1]
                data['signal'].iloc[i] = "Long carry"
                
                if data['Close'][i] < data['supertrend'][i]: #supertrend crossover exit
                    PL = round(data['Close'][i] - entry_price,2)
                    #print('{:<8} | Exit  Price: {:<8} | entry: {:<8} | Target: {:<8} | signal hit: {:<8}\n'.format(str(data.index.time[i]),data['Close'][i],entry_price,target,PL))

                    data['position'].iloc[i] = 0
                    data['trade_price'].iloc[i] = data['Close'][i]
                    data['signal'].iloc[i] = "signal Hit"
                    data['trade_return'].iloc[i] = PL

                    entry_price = 0
                    stoploss = 0
                    trailing_stoploss = 0
                    target = 0
                    continue

                if data['Close'][i] >= target: #target hit exit
                    PL = round(data['Close'][i] - entry_price,2)
                    #print('{:<8} | Exit  Price: {:<8} | entry: {:<8} | Target: {:<8} | target hit: {:<8}\n'.format(str(data.index.time[i]),data['Close'][i],entry_price,target,PL))

                    data['position'].iloc[i] = 0
                    data['trade_price'].iloc[i] = data['Close'][i]
                    data['signal'].iloc[i] = "target Hit"
                    data['trade_return'].iloc[i] = PL

                    entry_price = 0
                    stoploss = 0
                    trailing_stoploss = 0
                    target = 0
                    continue

                if data['Close'][i] <= stoploss: #stoploss hit exit
                    PL = round(data['Close'][i] - entry_price,2)
                    #print('{:<8} | Long Exit  Price: {:<8} | entry: {:<8} | Target: {:<8} | SL hit: {:<8}\n'.format(str(data.index.time[i]),data['Close'][i],entry_price,target,PL))

                    data['position'].iloc[i] = 0
                    data['trade_price'].iloc[i] = data['Close'][i]
                    data['signal'].iloc[i] = "SL Hit"
                    data['trade_return'].iloc[i] = PL

                    entry_price = 0
                    stoploss = 0
                    trailing_stoploss = 0
                    target = 0
                    continue

                if data['Close'][i] > trailing_stoploss: #updating the new stoploss
                    updated_stoploss = trailing_stoploss
                    stoploss = updated_stoploss
                    trailing_stoploss = trailing_stoploss + trailing_stoploss_pt
                    #print('{:<8} | carry price: {:<8} | entry: {:<8} | Target: {:<8} | new SL: {:<8} | TSL: {:<8}'.format(str(data.index.time[i]),data['Close'][i],entry_price,target,updated_stoploss,trailing_stoploss))

                #if data['Close'][i] < trailing_stoploss: # carry forward the sl target and Trailing stoploss
                    #print('{:<8} | carry price: {:<8} | entry: {:<8} | Target: {:<8} | SL    : {:<8} | TSL: {:<8}'.format(str(data.index.time[i]),data['Close'][i],entry_price,target,stoploss,trailing_stoploss))

            if data['Close'][i] > data['supertrend'][i] and data['position'][i] == 0: # entrying the new long position
                data['position'].iloc[i] = 1
                data['trade_price'].iloc[i] = data['Close'][i]
                data['signal'].iloc[i] = "Entry long"

                ## long entry price
                entry_price = data['Close'][i]
                stoploss = data['Close'][i] - stoploss_pt
                target = entry_price + target_pt 
                trailing_stoploss = entry_price + trailing_stoploss_pt
                #print('{:<8} | Long Entry Price: {:<8} | Target: {:<8} | SL: {:<8} | TSL: {:<8}'.format(str(data.index.time[i]),entry_price,target,stoploss,trailing_stoploss))

    #############################################################################################################        
            ##### short position
            if data['position'][i-1] == -1:
                data['position'].iloc[i] = data['position'].iloc[i-1]
                data['signal'].iloc[i] = "short carry"

                if data['Close'][i] > data['supertrend'][i]: #signal hit
                    short_PL = round(short_entry_price-data['Close'][i],2)
                    #print('{:<8} | short Exit Price: {:<8} | entry: {:<8} | Target: {:<8} | signal hit: {:<8}\n'.format(str(data.index.time[i]),data['Close'][i],short_entry_price,short_target,short_PL))

                    data['position'].iloc[i] = 0
                    data['trade_price'].iloc[i] = data['Close'][i]
                    data['signal'].iloc[i] = "short signal Hit"
                    data['trade_return'].iloc[i] = short_PL

                    short_entry_price = 0
                    short_stoploss = 0
                    short_trailing_stoploss = 0
                    short_target = 0
                    continue

                if data['Close'][i] <= short_target: #target hit
                    short_PL = round(short_entry_price - data['Close'][i],2)
                    #print('{:<8} | short Exit Price: {:<8} | entry: {:<8} | Target: {:<8} | target hit: {:<8}\n'.format(str(data.index.time[i]),data['Close'][i],short_entry_price,short_target,short_PL))

                    data['position'].iloc[i] = 0
                    data['trade_price'].iloc[i] = data['Close'][i]
                    data['signal'].iloc[i] = "short target Hit"
                    data['trade_return'].iloc[i] = short_PL

                    short_entry_price = 0
                    short_stoploss = 0
                    short_trailing_stoploss = 0
                    short_target = 0
                    continue

                if data['Close'][i] >= short_stoploss: #stoploss hit exit
                    short_PL = round(short_entry_price - data['Close'][i],2)
                    #print('{:<8} | short Exit Price: {:<8} | entry: {:<8} | Target: {:<8} | SL hit: {:<8}\n'.format(str(data.index.time[i]),data['Close'][i],short_entry_price,short_target,short_PL))

                    data['position'].iloc[i] = 0
                    data['trade_price'].iloc[i] = data['Close'][i]
                    data['signal'].iloc[i] = "short SL hit"
                    data['trade_return'].iloc[i] = short_PL

                    short_entry_price = 0
                    short_stoploss = 0
                    short_trailing_stoploss = 0
                    short_target = 0
                    continue

                if data['Close'][i] < short_trailing_stoploss: #updating the new stoploss
                    updated_short_stoploss = short_trailing_stoploss
                    short_stoploss = updated_short_stoploss
                    short_trailing_stoploss = short_trailing_stoploss - trailing_stoploss_pt
                    #print('{:<8} | carry price: {:<8} | entry: {:<8} | Target: {:<8} | new SL: {:<8} | TSL: {:<8}'.format(str(data.index.time[i]),data['Close'][i],short_entry_price,short_target,updated_short_stoploss,short_trailing_stoploss))

                #if data['Close'][i] > short_trailing_stoploss: # carry forward the sl target and trailing stoploss
                    #print('{:<8} | carry price: {:<8} | entry: {:<8} | Target: {:<8} | SL    : {:<8} | TSL: {:<8}'.format(str(data.index.time[i]),data['Close'][i],short_entry_price,short_target,short_stoploss,short_trailing_stoploss))


            if data['Close'][i] < data['supertrend'][i] and data['position'][i] == 0:
                data['position'].iloc[i] = -1
                data['trade_price'].iloc[i] = data['Close'][i]
                data['signal'].iloc[i] = "Entry short"

                ## short entry price
                short_entry_price = data['Close'][i]
                short_stoploss = data['Close'][i] + stoploss_pt
                short_target = short_entry_price - target_pt
                short_trailing_stoploss = short_entry_price - trailing_stoploss_pt
                #print('{:<8} | short Entry Price: {:<8} | Target: {:<8} | SL: {:<8} | TSL: {:<8}'.format(str(data.index.time[i]),short_entry_price,short_target,short_stoploss,short_trailing_stoploss))
        
    
    return data

In [16]:
#daywise dataframe
result_columns = ['date','PL']
result_dataframe = pd.DataFrame()
result_dataframe

""


In [17]:
nifty_path = "D:\\stock_analysis_Spyder\\28-06-2021 - live_ohlc\\Nifty 50.csv"
#nifty_path = "D:\\stock_analysis_Spyder\\28-06-2021 - live_ohlc\\NIFTY JUL 15700.0 CE.csv"

In [243]:
#nifty_path = "D:\\stock_analysis_Spyder\\NFO\\NIFTY 05 AUG21\\NIFTY 05 AUG21 15750.0 CE.csv"

In [26]:
raw_data = read_data(nifty_path,'5min')

In [27]:
dt = np.unique(raw_data.index.date)
dt

array([datetime.date(2021, 6, 29), datetime.date(2021, 6, 30),
       datetime.date(2021, 7, 5), datetime.date(2021, 7, 6),
       datetime.date(2021, 7, 7), datetime.date(2021, 7, 8),
       datetime.date(2021, 7, 9), datetime.date(2021, 7, 12),
       datetime.date(2021, 7, 13), datetime.date(2021, 7, 14),
       datetime.date(2021, 7, 15), datetime.date(2021, 7, 16),
       datetime.date(2021, 7, 19), datetime.date(2021, 7, 22),
       datetime.date(2021, 7, 23)], dtype=object)

In [28]:
#2min interval
for i,date_time in enumerate(dt):
    ind_df = strategy_technical_indicator(raw_data)
    df = ind_df[ind_df.index.date == date_time]
    row = df.shape[0]
    df = df[['Close','supertrend']][1:]
    start = str(df.index[0].time())
    end = str(df.index[-1].time())
    st_data = st_crossover(df,10,20,5)
    result = round(st_data['trade_return'].sum(),2)
    #print(i,date_time," = ",result," ",row)
    print("{:<3} | {:<8} | PL = {:<6} | From : {:<8} | To : {:<8}".format(i,str(date_time),result,start,end))
    result_dataframe['date'] = str(date_time)
    result_dataframe['PL'] = result
    #result_dataframe = result_dataframe.append(result)

0   | 2021-06-29 | PL = 13.6   | From : 12:10:00 | To : 13:55:00
1   | 2021-06-30 | PL = -6.9   | From : 12:00:00 | To : 13:10:00
2   | 2021-07-05 | PL = -21.95 | From : 12:50:00 | To : 15:45:00
3   | 2021-07-06 | PL = 87.3   | From : 09:40:00 | To : 15:25:00
4   | 2021-07-07 | PL = -16.85 | From : 12:15:00 | To : 15:45:00
5   | 2021-07-08 | PL = 15.95  | From : 09:20:00 | To : 15:30:00
6   | 2021-07-09 | PL = -66.65 | From : 09:20:00 | To : 15:20:00
7   | 2021-07-12 | PL = -43.95 | From : 09:30:00 | To : 15:35:00
8   | 2021-07-13 | PL = -18.85 | From : 09:35:00 | To : 15:35:00
9   | 2021-07-14 | PL = 23.85  | From : 09:30:00 | To : 15:30:00
10  | 2021-07-15 | PL = 4.75   | From : 09:15:00 | To : 10:00:00
11  | 2021-07-16 | PL = -48.75 | From : 09:20:00 | To : 15:30:00
12  | 2021-07-19 | PL = 16.35  | From : 09:20:00 | To : 15:35:00
13  | 2021-07-22 | PL = 33.45  | From : 09:10:00 | To : 13:35:00
14  | 2021-07-23 | PL = -29.4  | From : 13:30:00 | To : 15:30:00


In [22]:
#1min interval
for i,date_time in enumerate(dt):
    ind_df = strategy_technical_indicator(raw_data)
    df = ind_df[ind_df.index.date == date_time]
    row = df.shape[0]
    df = df[['Close','supertrend']][1:]
    start = str(df.index[0].time())
    end = str(df.index[-1].time())
    st_data = st_crossover(df,10,20,5)
    result = round(st_data['trade_return'].sum(),2)
    #print(i,date_time," = ",result," ",row)
    print("{:<3} | {:<8} | PL = {:<6} | From : {:<8} | To : {:<8}".format(i,str(date_time),result,start,end))
    result_dataframe['date'] = str(date_time)
    result_dataframe['PL'] = result
    #result_dataframe = result_dataframe.append(result)

0   | 2021-06-29 | PL = 24.65  | From : 12:10:00 | To : 13:57:00
1   | 2021-06-30 | PL = -14.2  | From : 12:00:00 | To : 13:13:00
2   | 2021-07-05 | PL = -25.4  | From : 12:48:00 | To : 15:49:00
3   | 2021-07-06 | PL = 13.2   | From : 09:40:00 | To : 15:25:00
4   | 2021-07-07 | PL = -9.85  | From : 12:11:00 | To : 15:49:00
5   | 2021-07-08 | PL = 44.3   | From : 09:17:00 | To : 15:31:00
6   | 2021-07-09 | PL = -53.9  | From : 09:18:00 | To : 15:22:00
7   | 2021-07-12 | PL = 78.5   | From : 09:28:00 | To : 15:37:00
8   | 2021-07-13 | PL = 7.85   | From : 09:33:00 | To : 15:36:00
9   | 2021-07-14 | PL = 5.05   | From : 09:28:00 | To : 15:31:00
10  | 2021-07-15 | PL = -14.85 | From : 09:13:00 | To : 10:02:00
11  | 2021-07-16 | PL = -55.65 | From : 09:17:00 | To : 15:31:00
12  | 2021-07-19 | PL = 20.6   | From : 09:18:00 | To : 15:35:00
13  | 2021-07-22 | PL = -20.05 | From : 09:06:00 | To : 13:35:00
14  | 2021-07-23 | PL = -56.9  | From : 13:28:00 | To : 15:30:00


In [223]:
result_dataframe

,PL,date


In [237]:
st_data['trade_return'][st_data['trade_return']>0]

time_stamp
2021-07-23 13:44:00    9.40
2021-07-23 13:58:00    3.05
2021-07-23 14:27:00    0.85
2021-07-23 14:41:00    9.50
2021-07-23 14:52:00    1.35
Name: trade_return, dtype: float64

In [238]:
st_data['trade_return'][st_data['trade_return']<0]

time_stamp
2021-07-23 14:00:00    -7.05
2021-07-23 14:08:00   -14.90
2021-07-23 14:14:00    -5.05
2021-07-23 14:17:00    -5.25
2021-07-23 14:21:00    -2.60
2021-07-23 14:43:00    -9.00
2021-07-23 14:45:00    -3.00
Name: trade_return, dtype: float64